## **Importing Necessary Packages**

In [1]:
# Import required packages
import numpy as np
import pandas as pd
import fr_core_news_sm
import en_core_web_sm
import tensorflow as tf

## **Load the Dataset**

In [2]:
# Loading data
doc = pd.read_csv("https://go.aws/38ECHUB", delimiter="\t", header=None)
doc.head()

,0,1
0,Go.,Va !
1,Hi.,Salut !
2,Run!,Cours !
3,Run!,Courez !
4,Wow!,Ça alors !


In [3]:
print(doc.shape)
# We are taking 5000 words from the dataset to fasten the training process
doc = doc.sample(5000)
# Loading of the entire corpus of French and English sentences
fr_corpus = " ".join(doc.iloc[:, 1].to_list())
en_corpus = " ".join(doc.iloc[:, 0].to_list())

(160538, 2)


## **Data Preprocessing**

In [4]:
# Loading both corpora into spacy
nlp_fr = fr_core_news_sm.load()
nlp_fr.max_length = len(fr_corpus)

nlp_en = en_core_web_sm.load()
nlp_en.max_length = len(en_corpus)

fr_doc = nlp_fr(fr_corpus)
en_doc = nlp_en(en_corpus)

In [5]:
# Tokenization of each sentence via spacy
doc["fr_tokens"] = doc.iloc[:, 1].apply(nlp_fr.tokenizer)
doc["en_tokens"] = doc.iloc[:, 0].apply(nlp_en.tokenizer)

In [6]:
doc.tail()

,0,1,fr_tokens,en_tokens
9502,Can they see me?,Sont-ils en mesure de me voir ?,"(Sont, -ils, en, mesure, de, me, voir, ?)","(Can, they, see, me, ?)"
73015,We need to get out of here.,Il nous faut sortir de là.,"(Il, nous, faut, sortir, de, là, .)","(We, need, to, get, out, of, here, .)"
58346,Many people were injured.,De nombreuses personnes ont été blessées.,"(De, nombreuses, personnes, ont, été, blessées...","(Many, people, were, injured, .)"
5566,I'm just lazy.,Je suis juste paresseuse.,"(Je, suis, juste, paresseuse, .)","(I, 'm, just, lazy, .)"
130073,What time did you wake up this morning?,À quelle heure t'es-tu réveillée ce matin ?,"(À, quelle, heure, t', es, -, tu, réveillée, c...","(What, time, did, you, wake, up, this, morning..."


In [7]:
# Creation of a set() that will take all the unique tokens from our text corpus
en_tokens = [token.text for token in en_doc]
en_vocabulary_set= set(en_tokens)
en_vocab_size = len(en_vocabulary_set)
# Same thing for French
fr_tokens = [token.text for token in fr_doc]
fr_vocabulary_set= set(fr_tokens)
fr_vocab_size = len(fr_vocabulary_set)
print("Eng vocab size:",en_vocab_size,"French vocab size:", fr_vocab_size)
# Creation of an id for each token
all_en_tokens = {en_token: i + 1 for i, en_token in enumerate(en_vocabulary_set)}
all_fr_tokens = {fr_token: i + 1 for i, fr_token in enumerate(fr_vocabulary_set)}
# RQ: We take at i+1 to leave the value 0 for the creation of the padded_sequences

Eng vocab size: 3508 French vocab size: 4911


In [8]:
# Creation of functions that will create a vector of indices for each of the token sequences
def en_tokens_to_index(tokens):
    return [all_en_tokens[token.text] for token in tokens]

def fr_tokens_to_index(tokens):
    return [all_fr_tokens[token.text] for token in tokens]

In [9]:
# Transformation of tokens into indices
doc["fr_indices"] = doc["fr_tokens"].apply(fr_tokens_to_index)
doc["en_indices"] = doc["en_tokens"].apply(en_tokens_to_index)
doc.tail()

,0,1,fr_tokens,en_tokens,fr_indices,en_indices
9502,Can they see me?,Sont-ils en mesure de me voir ?,"(Sont, -ils, en, mesure, de, me, voir, ?)","(Can, they, see, me, ?)","[3100, 3595, 543, 3582, 4748, 1909, 143, 2940]","[1811, 2336, 2846, 1361, 2105]"
73015,We need to get out of here.,Il nous faut sortir de là.,"(Il, nous, faut, sortir, de, là, .)","(We, need, to, get, out, of, here, .)","[1499, 4605, 1974, 3888, 4748, 1787, 2076]","[934, 1386, 2434, 3031, 2089, 2180, 1347, 1486]"
58346,Many people were injured.,De nombreuses personnes ont été blessées.,"(De, nombreuses, personnes, ont, été, blessées...","(Many, people, were, injured, .)","[2603, 2535, 3994, 3152, 382, 4207, 2076]","[3397, 2999, 2067, 2228, 1486]"
5566,I'm just lazy.,Je suis juste paresseuse.,"(Je, suis, juste, paresseuse, .)","(I, 'm, just, lazy, .)","[3161, 1686, 1920, 556, 2076]","[2594, 1886, 1153, 2663, 1486]"
130073,What time did you wake up this morning?,À quelle heure t'es-tu réveillée ce matin ?,"(À, quelle, heure, t', es, -, tu, réveillée, c...","(What, time, did, you, wake, up, this, morning...","[63, 1705, 399, 3466, 1803, 2835, 423, 1259, 3...","[1041, 265, 3042, 922, 472, 223, 1398, 323, 2105]"


In [10]:
# Use of Keras to create token sequences of the same length
padded_fr_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["fr_indices"], padding="post")
padded_en_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["en_indices"], padding="post")

In [11]:
# Visualization of the shape of one of the tensors
print("Padded french indices:",padded_fr_indices.shape,"Padded english shape:",padded_en_indices.shape)

Padded french indices: (5000, 33) Padded english shape: (5000, 32)


In [12]:
doc["fr_indices"].apply(len).max(),doc["en_indices"].apply(len).max()

(33, 32)

In [13]:
# Application of the categorization of the target variable
binarized_en_indices = tf.keras.utils.to_categorical(padded_en_indices, num_classes=en_vocab_size+1)
binarized_en_indices.shape

(5000, 32, 3509)

In [14]:
# Creation of tf.data.Dataset for each of the French and English tensors
fr_ds = tf.data.Dataset.from_tensor_slices(padded_fr_indices)
en_ds = tf.data.Dataset.from_tensor_slices(binarized_en_indices)

In [15]:
# Create a complete tensorflow dataset
tf_ds = tf.data.Dataset.zip((fr_ds, en_ds))
next(iter(tf_ds))

(<tf.Tensor: shape=(33,), dtype=int32, numpy=
 array([3840, 1297,  423, 3903, 3175, 1261, 2076,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       dtype=int32)>,
 <tf.Tensor: shape=(32, 3509), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)

In [16]:
# Shuffle & Batch
BATCH_SIZE = 32

tf_ds = tf_ds.shuffle(len(doc)).batch(BATCH_SIZE)

In [17]:
# Train Test Split
TAKE_SIZE = int(0.7 * len(doc) / BATCH_SIZE)

train_data = tf_ds.take(TAKE_SIZE)
test_data = tf_ds.skip(TAKE_SIZE)

# **Creating the Model to train the data**

In [18]:
# Create the model
model = tf.keras.Sequential([
                  # Input Word Embedding layer
                  tf.keras.layers.Embedding(fr_vocab_size + 1, 64, mask_zero=True),

                  # LSTM Bidirectional layer
                  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),

                  # LSTM Bidirectionnal new layer
                  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),

                  # Repeat Vector
                  tf.keras.layers.RepeatVector(binarized_en_indices.shape[1]),

                  # LSTM new layer
                  tf.keras.layers.LSTM(32, return_sequences=True),

                  # Output layer with number of output neurons equal to class number with softmax function
                  tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(en_vocab_size+1, activation="softmax"))

])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          314368    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 repeat_vector (RepeatVecto  (None, 32, 128)           0         
 r)                                                              
                                                                 
 lstm_2 (LSTM)               (None, 32, 32)            20608     
                                                        

In [19]:
# "Random" prediction to test our model
input_text, output_text = next(iter(train_data))
print(input_text.numpy().shape)
print(model.predict(input_text).shape)
print(output_text.numpy().shape)

(32, 33)
1/1 [==============================] - 6s 6s/step
(32, 32, 3509)
(32, 32, 3509)


In [20]:
# Let's create a learning rate schedule to decrease the learning rate as we train the model
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1090,
    decay_rate=0.96,
    staircase=True)

# Using a simple compiler with an Adam optimizer to compute our gradients
optimizer= tf.keras.optimizers.Adam(
    learning_rate = lr_schedule
)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

## **Training the Model**

In [21]:
# Training on 200 epochs
history_1 = model.fit(train_data,
                      validation_data=test_data,
                      epochs=200)

Epoch 1/200
109/109 [==============================] - 39s 169ms/step - loss: 5.0429 - categorical_accuracy: 0.7545 - val_loss: 2.4385 - val_categorical_accuracy: 0.7598
Epoch 2/200
109/109 [==============================] - 7s 60ms/step - loss: 2.0209 - categorical_accuracy: 0.7620 - val_loss: 1.8513 - val_categorical_accuracy: 0.7641
Epoch 3/200
109/109 [==============================] - 6s 56ms/step - loss: 1.7565 - categorical_accuracy: 0.7630 - val_loss: 1.6572 - val_categorical_accuracy: 0.7630
Epoch 4/200
109/109 [==============================] - 5s 47ms/step - loss: 1.6168 - categorical_accuracy: 0.7623 - val_loss: 1.5545 - val_categorical_accuracy: 0.7640
Epoch 5/200
109/109 [==============================] - 6s 53ms/step - loss: 1.5284 - categorical_accuracy: 0.7637 - val_loss: 1.4877 - val_categorical_accuracy: 0.7660
Epoch 6/200
109/109 [==============================] - 5s 47ms/step - loss: 1.4777 - categorical_accuracy: 0.7674 - val_loss: 1.4505 - val_categorical_accurac

In [26]:
# Training on 500 more epochs
history_2 = model.fit(train_data,
                      validation_data=test_data,
                      epochs=500)

Epoch 1/500
109/109 [==============================] - 6s 53ms/step - loss: 0.7216 - categorical_accuracy: 0.8557 - val_loss: 0.7112 - val_categorical_accuracy: 0.8577
Epoch 2/500
109/109 [==============================] - 5s 46ms/step - loss: 0.7211 - categorical_accuracy: 0.8555 - val_loss: 0.7100 - val_categorical_accuracy: 0.8569
Epoch 3/500
109/109 [==============================] - 6s 52ms/step - loss: 0.7166 - categorical_accuracy: 0.8564 - val_loss: 0.6998 - val_categorical_accuracy: 0.8576
Epoch 4/500
109/109 [==============================] - 6s 54ms/step - loss: 0.7133 - categorical_accuracy: 0.8567 - val_loss: 0.7363 - val_categorical_accuracy: 0.8530
Epoch 5/500
109/109 [==============================] - 5s 45ms/step - loss: 0.7031 - categorical_accuracy: 0.8588 - val_loss: 0.7302 - val_categorical_accuracy: 0.8538
Epoch 6/500
109/109 [==============================] - 6s 59ms/step - loss: 0.7065 - categorical_accuracy: 0.8584 - val_loss: 0.7016 - val_categorical_accuracy:

## **Testing the Model**

In [36]:
# Taking the words randomly for translation
for input_text, translation in test_data.take(1):
    pred = np.argmax(model.predict(input_text), axis=-1)
# CAUTION: it's possible because there is only one key for each value
indice_to_fr_token = dict(zip(all_fr_tokens.values(), all_fr_tokens.keys()))
indice_to_en_token = dict(zip(all_fr_tokens.values(), all_en_tokens.keys()))

# Testing the translation

for input_text, translation in test_data.take(1):
    pred = np.argmax(model.predict(input_text), axis=-1)

# Input in French to translate
print("Input Sentence:", end=" ")
for indice in input_text[2]:
    if indice == 0:
        break
    print(indice_to_fr_token[indice.numpy()], end=" ")

# Correct translation for the input
print("\nTrue Translation:", end=" ")
for indice in np.argmax(translation, axis=-1)[2]:
    if indice == 0:
        break
    print(indice_to_en_token[indice], end=" ")

# Predicted sentence in English
print("\nModel Translation:", end=" ")
for indice in pred[2]:
    if indice == 0:
        break
    print(indice_to_en_token[indice], end=" ")

1/1 [==============================] - 0s 29ms/step
Input Sentence: Arrête de crier . 
True Translation: Stop shouting . 
Model Translation: Stop shouting . 

**Note**: We have only trained this model on 5000 of the available 160000 words because of the lack of hardware. We were able to achieve an accuracy of 90%. However, the accuracy can be improved when we train the model to the full dataset and adjust some hyperparameters like the number of layers in the neural network and so on